In [1]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


### Load document into backend

In [2]:
DOCUMENT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
)

result: Result[str, str] = backend.load_project(DOCUMENT_PATH)
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

LoadProject: path = /data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz


2024-09-03 12:41:01.805 | INFO     | __main__:<module>:8 - loaded document /data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz successfully


loaded project in 38.7785 sec


### Backend playground

In [5]:
from mynd.api.camera import StereoCollection
from mynd.api.identifier import Identifier

from mynd.camera import CameraCalibration
from mynd.io.file_database import H5Database, create_file_database, load_file_database

from mynd.geometry import (
    PixelMap,
    RectificationTransforms,
    RectificationResult,
    compute_rectifying_camera_transforms,
    compute_rectifying_image_transforms,
    compute_stereo_rectification,
    rectify_image_pair,
)

# NOTE: Integrate camera readers and writers into IO library eventually
from mynd.io.camera_io import write_rectification_results_to_file_database


def process_stereo_collection(
    identifier: Identifier, collection: StereoCollection
) -> None:
    """Process a collection of stereo data."""

    rectification: RectificationResult = compute_stereo_rectification(
        collection.calibrations
    )

    logger.info(f"Key:                 {identifier}")
    logger.info(f"Calibrations:        {len(collection.calibrations)}")
    logger.info(f"Image loaders:       {len(collection.image_loaders)}")

    DATABASE_PATH: Path = Path(
        f"/data/kingston_snv_01/test/{identifier.label}_calibration.h5"
    )

    if DATABASE_PATH.exists():
        result: Result[H5Database, str] = load_file_database(DATABASE_PATH)
    else:
        result: Result[H5Database, str] = create_file_database(DATABASE_PATH)

    if result.is_err():
        logger.error(f"Database error: {result}")

    database: H5Database = result.ok()

    result: Result[None, str] = write_rectification_results_to_file_database(
        database,
        group_name="stereo",
        rectification=rectification,
    )

    if result.is_err():
        logger.error(result.err())

    database.get("stereo").visit(lambda item: logger.info(item))


def main():
    """TODO"""
    result: Result = backend.camera_services.request_stereo_data()

    data: dict[Identifier, list[StereoCollection]] = result.unwrap()

    for identifier, collections in data.items():

        for collection in collections:
            process_stereo_collection(identifier, collection)


# ---------- Invoke main function ----------
main()

2024-09-03 12:41:54.428 | INFO     | __main__:process_stereo_collection:30 - Key:                 Identifier(key=0, label='r23685bc_20100605_021022')
2024-09-03 12:41:54.429 | INFO     | __main__:process_stereo_collection:31 - Calibrations:        2
2024-09-03 12:41:54.429 | INFO     | __main__:process_stereo_collection:32 - Image loaders:       2728
2024-09-03 12:41:54.452 | INFO     | __main__:<lambda>:57 - calibrations
2024-09-03 12:41:54.453 | INFO     | __main__:<lambda>:57 - calibrations/raw
2024-09-03 12:41:54.453 | INFO     | __main__:<lambda>:57 - calibrations/raw/first
2024-09-03 12:41:54.453 | INFO     | __main__:<lambda>:57 - calibrations/raw/first/camera_matrix
2024-09-03 12:41:54.453 | INFO     | __main__:<lambda>:57 - calibrations/raw/first/distortion
2024-09-03 12:41:54.454 | INFO     | __main__:<lambda>:57 - calibrations/raw/first/height
2024-09-03 12:41:54.454 | INFO     | __main__:<lambda>:57 - calibrations/raw/first/location
2024-09-03 12:41:54.454 | INFO     | __ma